# Download

Berikut adalah contoh script untuk mengunduh data GDELT dengan country code ID. Selanjutnya dari csv bisa dipindah ke mysql

In [4]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits
file_list = [x for x in link_list if str.isdigit(x[0:4])]
# file_list = file_list[85:]
print(file_list)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib.request
import zipfile
import glob
import operator
import pandas as pd

local_path = 'C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/'

fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(infilecounter, ' - ', compressed_file)

    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, ')
#         urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
#                            filename=local_path + compressed_file)
        
        resp = requests.get(url=gdelt_base_url + compressed_file)

        # assuming the subdirectory tempdata has been created:
        zfile = open(local_path + compressed_file, 'wb')
        zfile.write(resp.content)
        zfile.close()

    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')

    # parse each of the csv files in the working directory,
    print('parsing,'),

    for infile_name in glob.glob(local_path + 'tmp/*'):
        outfile_name = local_path + 'country/' + fips_country_code + '%04i.tsv' % outfilecounter
        # open the infile and outfile
                
        with open(infile_name, mode='r', encoding="utf8") as infile, open(outfile_name, mode='w', encoding="utf8") as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
    infilecounter += 1
    print('done')

['20181101.export.CSV.zip', '20181031.export.CSV.zip', '20181030.export.CSV.zip', '20181029.export.CSV.zip', '20181028.export.CSV.zip', '20181027.export.CSV.zip', '20181026.export.CSV.zip', '20181025.export.CSV.zip', '20181024.export.CSV.zip', '20181023.export.CSV.zip', '20181022.export.CSV.zip', '20181021.export.CSV.zip', '20181020.export.CSV.zip', '20181019.export.CSV.zip', '20181018.export.CSV.zip', '20181017.export.CSV.zip', '20181016.export.CSV.zip', '20181015.export.CSV.zip', '20181014.export.CSV.zip', '20181013.export.CSV.zip', '20181012.export.CSV.zip', '20181011.export.CSV.zip', '20181010.export.CSV.zip', '20181009.export.CSV.zip', '20181008.export.CSV.zip', '20181007.export.CSV.zip', '20181006.export.CSV.zip', '20181005.export.CSV.zip', '20181004.export.CSV.zip', '20181003.export.CSV.zip', '20181002.export.CSV.zip', '20181001.export.CSV.zip', '20180930.export.CSV.zip', '20180929.export.CSV.zip', '20180928.export.CSV.zip', '20180927.export.CSV.zip', '20180926.export.CSV.zip', 

parsing,
done
1  -  20181031.export.CSV.zip
extracting,
parsing,
done
2  -  20181030.export.CSV.zip
extracting,
parsing,
done
3  -  20181029.export.CSV.zip
extracting,
parsing,
done
4  -  20181028.export.CSV.zip
extracting,
parsing,
done
5  -  20181027.export.CSV.zip
extracting,
parsing,
done
6  -  20181026.export.CSV.zip
extracting,
parsing,
done
7  -  20181025.export.CSV.zip
extracting,
parsing,
done
8  -  20181024.export.CSV.zip
extracting,
parsing,
done
9  -  20181023.export.CSV.zip
extracting,
parsing,
done
10  -  20181022.export.CSV.zip
extracting,
parsing,
done
11  -  20181021.export.CSV.zip
extracting,
parsing,
done
12  -  20181020.export.CSV.zip
extracting,
parsing,
done
13  -  20181019.export.CSV.zip
extracting,
parsing,
done
14  -  20181018.export.CSV.zip
extracting,
parsing,
done
15  -  20181017.export.CSV.zip
extracting,
parsing,
done
16  -  20181016.export.CSV.zip
extracting,
parsing,
done
17  -  20181015.export.CSV.zip
extracting,
parsing,
done
18  -  20181014.export.CSV

BadZipFile: File is not a zip file

# Scraping GDELT
Berikut adalah kode contoh untuk melakukan scraping dengan library newspaper. Skrip ini akan berjalan jika data dari GDELT telah tersimpan ke MySQL

In [38]:
from newspaper import Article
import pymysql.cursors
import sys

offset = int(input("Insert minimum limit (minimum is 0): "))
limit = int(input("Insert maximum limit (minimum is 1): "))
# year = sys.argv[3]

# Connect to the database
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}

with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') LIMIT "+str(offset)+","+str(limit)
    cursor.execute(sql)
    result = cursor.fetchall()
#     print(result)
    for r in result:
        try:
            r["URL"] = r["SOURCEURL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            print(article)
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

Insert minimum limit (minimum is 0): 0
Insert maximum limit (minimum is 1): 50
https://www.goldcoastbulletin.com.au/the-13-minutes-before-tragedy-struck-how-the-lion-air-disaster-unfolded/news-story/28de53a23285acd177d9f5386b6731fb?nk=c468b881f39c524cdee34dbbe0916e22-1540856817
https://www.channelnewsasia.com/news/asia/lion-air-crash-victims-families-crying-frustrated-indonesia-10877680
https://www.unmultimedia.org/avlibrary/asset/2291/2291761/
https://www.channelnewsasia.com/news/asia/all-189-on-board-feared-dead-lion-air-crash-indonesia-10877728
https://www.goldcoastbulletin.com.au/the-13-minutes-before-tragedy-struck-how-the-lion-air-disaster-unfolded/news-story/28de53a23285acd177d9f5386b6731fb?nk=c468b881f39c524cdee34dbbe0916e22-1540856817
https://www.goldcoastbulletin.com.au/the-13-minutes-before-tragedy-struck-how-the-lion-air-disaster-unfolded/news-story/28de53a23285acd177d9f5386b6731fb?nk=c468b881f39c524cdee34dbbe0916e22-1540856817
https://www.goldcoastbulletin.com.au/the-13-mi

https://www.bigcountryhomepage.com/news/national/lion-air-crash-renews-concerns-about-indonesia-airlines/1560537838
https://www.adn.com/nation-world/2018/10/29/indonesia-plane-crash-search-finds-remains-debris-at-sea/
https://www.bigcountryhomepage.com/news/national/lion-air-crash-renews-concerns-about-indonesia-airlines/1560537838
https://www.ksl.com/article/46416809/former-australian-prime-minister-warns-of-indonesia-reaction
https://www.ksl.com/article/46416809/former-australian-prime-minister-warns-of-indonesia-reaction
https://www.ksl.com/article/46416809/former-australian-prime-minister-warns-of-indonesia-reaction
https://www.ksl.com/article/46416809/former-australian-prime-minister-warns-of-indonesia-reaction


Berikut adalah contoh untuk melakukan pickling data GDELT

In [5]:
import glob
import pandas as pd

local_path = 'C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/'

fips_country_code = 'ID'
# Get the GDELT field names from a helper file
colnames = pd.read_excel(local_path+'CSV.header.fieldids.xlsx', sheet_name='Sheet1',
                         index_col='Column ID', usecols=1)['Field Name']

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path + 'country/' + fips_country_code + '*')
DFlist = []
for active_file in files:
    print(active_file)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path + 'backupsmall' + fips_country_code + '.pickle')

# once everythin is safely stored away, remove the temporary files
# for active_file in files:
#     os.remove(active_file)

C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0000.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0001.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0002.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0003.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0004.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0005.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0006.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0007.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0008.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0009.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0010.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0011.tsv
C:/Users/a.haris/Documents/2018/Digital 

C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0112.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0113.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0114.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0115.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0116.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0117.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0118.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0119.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0120.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0121.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0122.tsv
C:/Users/a.haris/Documents/2018/Digital Talent/Project/GDELT/country\ID0123.tsv
C:/Users/a.haris/Documents/2018/Digital 

In [45]:
import bz2
import pickle

infile = open(local_path + 'backupsmallID.pickle','rb')
new_dict = pickle.load(infile)
new_dict.to_csv(local_path + fips_country_code + '.csv')
new_dict

,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,
798469840,20181029,201810,2018,2018.8192,IGOEUREEC,EUROPEAN COMMISSION,EUR,EEC,NaN,NaN,...,9179538,4,"Juanda, Jawa Timur, Indonesia",ID,ID08,-7.3855,112.764,9179538,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...
798469934,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,-2679652,4,"Jakarta, Jakarta Raya, Indonesia",ID,ID04,-6.17444,106.829,-2679652,20181030,https://www.channelnewsasia.com/news/asia/lion...
798470075,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,ID,1,United States,US,US,39.828175,-98.5795,US,20181030,https://www.unmultimedia.org/avlibrary/asset/2...
798470108,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,-2690876,4,"Pangkal Pinang, Indonesia (general), Indonesia",ID,ID00,-2.13333,106.133,-2690876,20181030,https://www.channelnewsasia.com/news/asia/all-...
798470109,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,-2679652,4,"Jakarta, Jakarta Raya, Indonesia",ID,ID04,-6.17444,106.829,-2679652,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...
798470110,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,-2690876,4,"Jakarta, Jakarta Raya, Indonesia",ID,ID04,-6.17444,106.829,-2679652,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...
798470111,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,-2690876,1,Australia,AS,AS,-25,135,AS,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...
798470112,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,10205777,4,"Bali, Jawa Timur, Indonesia",ID,ID08,-7.1046,112.337,10205777,20181030,https://www.ksl.com/article/46416809/former-au...
798470114,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,NaN,...,9116251,4,"Karawang, Jawa Barat, Indonesia",ID,ID30,-6.4403,106.471,9116251,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...
